In [21]:
import gymnasium as gym
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import matplotlib as plt
%matplotlib inline

In [42]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95   # Discount factor
        self.epsilon = 1.0  # Exploration rate (using epsilon greedy action-selection method here)
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9
        self.alpha = 0.001 # learning rate
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.alpha))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > 2000:
            del self.memory[0]

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [47]:
env = gym.make('FrozenLake-v1')
state_size = env.observation_space.n
action_size = env.action_space.n
# Assume DQNAgent is correctly implemented
agent = DQNAgent(state_size, action_size)

batch_size = 100
episodes = 1000

def state_to_index(state, shape):
    index = 0
    for i in range(len(shape)):
        index += state[i] * np.prod(shape[i+1:])
    return index

for e in range(episodes):
    state = env.reset()
    state_reshaped = state_to_index(state, env.observation_space.shape)
    for time in range(500):
        action = agent.act(state_reshaped)
        next_state, reward, done, _, _ = env.step(action)
        reward = reward if not done else -10
        next_state_reshaped = state_to_index(next_state, env.observation_space.shape)  # Reshape to match input shape
        agent.remember(state_reshaped, action, reward, next_state_reshaped, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, episodes, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)




episode: 0/1000, score: 9, e: 1.0
episode: 1/1000, score: 14, e: 1.0
episode: 2/1000, score: 2, e: 1.0
episode: 3/1000, score: 8, e: 1.0
episode: 4/1000, score: 2, e: 1.0
episode: 5/1000, score: 5, e: 1.0
episode: 6/1000, score: 3, e: 1.0
episode: 7/1000, score: 8, e: 1.0
episode: 8/1000, score: 20, e: 1.0
episode: 9/1000, score: 1, e: 1.0
episode: 10/1000, score: 4, e: 1.0
episode: 11/1000, score: 2, e: 1.0
episode: 12/1000, score: 2, e: 1.0
episode: 13/1000, score: 1, e: 1.0
episode: 14/1000, score: 3, e: 1.0


IndexError: tuple index out of range